<a href="https://colab.research.google.com/github/5quidL0rd/Unsloth-project/blob/main/Unsloth_Haiku_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Step 1: Install Dependencies

%%capture
!pip install unsloth
!pip install kaggle

!pip install trl

!pip install peft

!pip install torch



!pip install --upgrade tensorflow

!pip install --upgrade transformers

!pip install --upgrade transformers peft datasets accelerate



In [2]:
%%capture
# Skip restarting message in Colab
import sys; modules = list(sys.modules.keys())
for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None

!pip install unsloth vllm
!pip install --upgrade pillow
# If you are running this notebook on local, you need to install `diffusers` too
# !pip install diffusers
# Temporarily install a specific TRL nightly version
!pip install git+https://github.com/huggingface/trl.git@e95f9fb74a3c3647b86f251b7e230ec51c64b72b

In [5]:
from unsloth import is_bfloat16_supported #type:ignore
import torch
from unsloth import FastLanguageModel
max_seq_length = 256 # Can increase for longer reasoning traces


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "meta-llama/Llama-3.2-1B-Instruct",
    max_seq_length = max_seq_length,
    load_in_4bit = True, # False for LoRA 16bit
    fast_inference = True, # Enable vLLM fast inference
    #max_lora_rank = lora_rank, #type:ignore
    gpu_memory_utilization = 0.6, # Reduce if out of memory
    device_map="auto",
)

model = FastLanguageModel.get_peft_model(
    model,
    r = 8, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ], # Remove QKVO if out of memory
    lora_alpha = 8,
    use_gradient_checkpointing = "unsloth", # Enable long context finetuning
    random_state = 3407,
)



==((====))==  Unsloth 2025.2.5: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/llama-3.2-1b-instruct-unsloth-bnb-4bit with actual GPU utilization = 59.59%
Unsloth: Your GPU has CUDA compute capability 7.5 with VRAM = 14.74 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 256. Num Sequences = 192.
Unsloth: vLLM's KV Cache can use up to 7.64 GB. Also swap space = 2 GB.
WARNING 02-10 21:28:00 config.py:2386] Casting torch.bfloat16 to torch.float16.
INFO 02-10 21:28:17 config.py:542] This model supports multiple tasks: {'embed', 'score', 'reward', 'generate', 'classify'}. D

tokenizer_config.json:   0%|          | 0.00/54.7k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

...

INFO 02-10 21:29:27 model_runner.py:1562] Graph capturing finished in 36 secs, took 0.35 GiB
INFO 02-10 21:29:27 llm_engine.py:431] init engine (profile, create kv cache, warmup model) took 47.67 seconds


tokenizer_config.json:   0%|          | 0.00/54.7k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

Unsloth 2025.2.5 patched 16 layers with 16 QKV layers, 16 O layers and 16 MLP layers.


In [6]:
# Step 2: Upload the Kaggle Dataset Manually for google collaborate
from google.colab import files

# Upload the CSV file manually from your local system (unzip beforehand for convenience)
uploaded = files.upload()




Saving all_haiku.csv to all_haiku.csv


In [29]:
import pandas as pd

# Load the CSV file, assuming the correct headers are in the second row (index 1)
df = pd.read_csv('/content/all_haiku.csv', header=0)

# Display the first few rows to confirm the columns are correct now (highly recommend kaggle datasets are unpredictable)
print(df.head())


   Unnamed: 0                0                      1                 2  \
0           0    fishing boats              colors of       the rainbow   
1           1  ash wednesday--    trying to remember           my dream   
2           2     snowy morn--    pouring another cup   of black coffee   
3           3     shortest day           flames dance       in the oven   
4           4             haze  half the horse hidden  behind the house   

        source                                     hash  
0  tempslibres           FISHINGBOATSCOLORSOFTHERAINBOW  
1  tempslibres      ASHWEDNESDAYTRYINGTOREMEMBERMYDREAM  
2  tempslibres  SNOWYMORNPOURINGANOTHERCUPOFBLACKCOFFEE  
3  tempslibres          SHORTESTDAYFLAMESDANCEINTHEOVEN  
4  tempslibres     HAZEHALFTHEHORSEHIDDENBEHINDTHEHOUSE  


In [30]:
# Check the first few rows of the dataframe to understand the structure
print(df.head())

# Step 5: Rename Columns and Remove Irrelevant Columns
# Manually rename the columns (based on the dataset structure)

df.rename(columns={
    '0': 'First Line',
    '1': 'Second Line',  # Renaming based on actual dataset
    '2': 'Third Line',  # Renaming based on actual dataset
}, inplace=True)

# Check the column names after renaming
print(df.columns)

# Remove any irrelevant columns (such as 'tempslibres' or 'FISHINGBOATSCOLORSOFTHERAINBOW')
df = df[['First Line', 'Second Line', 'Third Line']]

# Check the cleaned dataframe to make sure everything is correct
print(df.head())




   Unnamed: 0                0                      1                 2  \
0           0    fishing boats              colors of       the rainbow   
1           1  ash wednesday--    trying to remember           my dream   
2           2     snowy morn--    pouring another cup   of black coffee   
3           3     shortest day           flames dance       in the oven   
4           4             haze  half the horse hidden  behind the house   

        source                                     hash  
0  tempslibres           FISHINGBOATSCOLORSOFTHERAINBOW  
1  tempslibres      ASHWEDNESDAYTRYINGTOREMEMBERMYDREAM  
2  tempslibres  SNOWYMORNPOURINGANOTHERCUPOFBLACKCOFFEE  
3  tempslibres          SHORTESTDAYFLAMESDANCEINTHEOVEN  
4  tempslibres     HAZEHALFTHEHORSEHIDDENBEHINDTHEHOUSE  
Index(['Unnamed: 0', 'First Line', 'Second Line', 'Third Line', 'source',
       'hash'],
      dtype='object')
        First Line            Second Line        Third Line
0    fishing boats           

In [31]:
# Step 6: Prepare the Dataset (Text Preprocessing)
# Ensure all columns are strings before concatenating
df['First Line'] = df['First Line'].fillna('').astype(str)
df['Second Line'] = df['Second Line'].fillna('').astype(str)
df['Third Line'] = df['Third Line'].fillna('').astype(str)

# Combine the 'First Line', 'Second Line', and 'Third Line' to create a full haiku for each row
poems = df['First Line'] + "\n" + df['Second Line'] + "\n" + df['Third Line']

# Clean the text by stripping whitespace and removing any empty haikus
poems = poems[poems.str.strip().apply(lambda x: len(x) > 0)].tolist()

# Join all the haikus into one large string, separated by new lines
poems_text = "\n".join(poems)

# Check the first few poems to ensure it's working
print(poems[:25])



['fishing boats\ncolors of\nthe rainbow', 'ash wednesday--\ntrying to remember \nmy dream', 'snowy morn--\npouring another cup\nof black coffee', 'shortest day\nflames dance\nin the oven', 'haze\nhalf the horse hidden\nbehind the house', 'low sun\nthe lady in red\non high heels', 'advent\nthe passing stranger\nfarts', 'tarn\na bubble in\nthe ice', 'snowflakes\nnew asphalt\nin the holes', "Crystal Night'\n   gusts of rain\n      outside", 'rain\nthe sound of a horse galloping\nthrough leaves', 'winter stars\nsuddenly a whiff\nof perfume', 'hungry\nhalf of the moon\nhidden', 'rain\nanother leaf\ndown', 'sharia\nthe sound of one hand\nclapping', 'the sound of geese\ndrowned by the sound of the train\nthis morning', 'autumn sun\nmy shadow over\ntombstones', 'fly fishing;\nthe sound of the wind\nin the reel', 'december\na long shadow\njoins another', 'end of path\nsnowflakes melting\non the pond', 'morning frost\nshe leaves\nfirst', 'evening walk\nsmell of tar between\npines', 'dachau\na bl

In [33]:
# Step 7: Prepare the Dataset for Fine-Tuning (Formatting Prompts)
# Define prompt format for model training
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

from transformers import AutoTokenizer #type: ignore

#tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")

# Formatting function for haiku dataset
EOS_TOKEN = tokenizer.eos_token  # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["First Line"]
    inputs = examples["Second Line"]
    outputs = examples["Third Line"]
    texts = []
    for instr, inp, out in zip(instructions, inputs, outputs):
        # Add EOS_TOKEN to end of each prompt
        text = alpaca_prompt.format(instr, inp, out) + EOS_TOKEN
        texts.append(text)
    return {"text": texts}

# Apply the formatting to the dataset
from datasets import Dataset #type: ignore
from datasets import load_dataset

# Ensure all values are strings and replace NaN with empty strings
df['First Line'] = df['First Line'].fillna('').astype(str)
df['Second Line'] = df['Second Line'].fillna('').astype(str)
df['Third Line'] = df['Third Line'].fillna('').astype(str)

# Create a dataset from the haiku text, ensuring all values are strings
dataset = Dataset.from_dict({
    "First Line": df['First Line'],
    "Second Line": df['Second Line'],
    "Third Line": df['Third Line'],
})

# Map the formatting function to the dataset

dataset = dataset.map(formatting_prompts_func, batched=True)
print(dataset[0])
print(f"Number of examples: {len(dataset)}")

# Now the dataset is ready for fine-tuning


Map:   0%|          | 0/144123 [00:00<?, ? examples/s]

{'First Line': 'fishing boats', 'Second Line': 'colors of', 'Third Line': 'the rainbow', 'text': 'Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nfishing boats\n\n### Input:\ncolors of\n\n### Response:\nthe rainbow<|eot_id|>'}
Number of examples: 144123


In [34]:
import os
import torch
import gc
from unsloth import FastLanguageModel #type: ignore
from trl import SFTTrainer #type: ignore
from transformers import TrainingArguments
from peft import get_peft_model, LoraConfig, TaskType
from accelerate import Accelerator

from trl import GRPOConfig, GRPOTrainer #type: ignore

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)
trainer_stats = trainer.train() #type: ignore




torch.distributed process group is initialized, but parallel_mode != ParallelMode.DISTRIBUTED. In order to use Torch DDP, launch your script with `python -m torch.distributed.launch
torch.distributed process group is initialized, but parallel_mode != ParallelMode.DISTRIBUTED. In order to use Torch DDP, launch your script with `python -m torch.distributed.launch


Map (num_proc=2):   0%|          | 0/144123 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 144,123 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 5,636,096


Step,Training Loss
1,2.131100
2,1.532200
3,1.611500
4,1.556900
5,1.485400
6,1.796300
7,1.488200
8,1.676900
9,1.475100
10,1.253200


In [35]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Give me a beautiful Haiku about nature", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

['<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nGive me a beautiful Haiku about nature\n\n### Input:\n\n\n### Response:\nthe trees the birds the wind the sun<|eot_id|>']